In [1]:
# Used to track processing time
import time

# Used to write a file with the current datetime
import datetime
import os

from textwrap import dedent

# Used to import LLM and set up mutli-agent crews
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

## Initialize the LLM

In [2]:
llm = Ollama(model="Llama3")

## Creating Agents

In [3]:
class CrewAgents:

    def variable_name_agent(self):
        return Agent(
            role='Senior Python Developer',
            goal=dedent(
                '''
                To review Python functions and ensure that variable names are
                following best practices. Variable names are concise, descriptive,
                and easy to understand.
                '''
            ),
            backstory=dedent(
                '''
                You're a Python developer who's been programming for decades. You're
                renowned for writing excellent code, and particularly for writing
                high-quality variable names.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def docstring_agent(self):
        return Agent(
            role='Senior Code Documentation Expert',
            goal='Write docstrings for Python functions.',
            backstory=dedent(
                '''
                You're a renowned code documentation expert, specifically in Python.
                You know how to write clean, concise docstrings that include all the
                information relevant to developers trying to understand a function.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )
    
    def quality_assurance_agent(self):
        return Agent(
            role="Senior Python Quality Assurance Tester",
            goal=dedent(
            '''
            To review Python code and ensure that it follows best practices.
            Double-check that variable names are accurate and concise and that
            the docstrings accurately reflect what the function actually does.
            '''
            ),
            backstory=dedent(
            '''
            You're a renowned quality assurance expert. You've been reviewing code
            for decades and ensure that code is flawless, with excellent variable
            names and docstrings.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )
    
    def code_writer_agent(self):
        return Agent(
            role="Senior Code Writer",
            goal=dedent(
            '''
            Take code and strip out everything except the code
            and docstrings.
            '''
            ),
            backstory=dedent(
            '''
            You do an amazing job of getting code and stripping it
            of everything that's not code or documentation.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )

## Create Tasks

In [4]:
class CrewTasks:

    def improve_variable_names(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the variable names in the Python code. Update
                the variable names so that they're accurate, concise, and
                easy to read and understand. Update the variable names only;
                don't change anything else about the functions, and do not
                add comments anywhere in the code.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with variable names that are
                accurate, concise, and easy to understand.
                '''
            ),
            agent=agent
        )

    def add_docstring(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Provide docstrings to the Python code provided below.
                Ensure the documentation is concise, accurate, and easy
                to read.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with a well-written, concise, and
                easy to understand docstring.
                '''
            ),
            agent=agent
        )

    def review_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review code and documentation and ensure it meets the
                highest standards of quality. The variable names are
                clear, concise, and accurate; the docstrings are
                accurate and descriptive, so that any developer reading
                them, even a junior developer, would understand exactly
                what they do and how to use them.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with excellent variable names and
                well-written, concise, and easy to understand docstring.
                '''
            ),
            agent=agent
        )

    def write_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the provided code and remove anything that
                is not code or docstrings. This information will
                then be written to a file that can be immediately used
                within a Python script.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                Code that consists ONLY of functions and their
                docstrings. All other information is stripped out.
                '''
            ),
            agent=agent
        )

## Read File
Reads a file of functions, each immediately proceeded by a comment.

These functions are read line by line and added to the `functions` list
for further processing.

In [5]:
# Define relative file path
test_suite_file_path = 'test_suite/test_functions_raw_practice.py'

# Save all captured functions to a list
functions = []

with open(test_suite_file_path, 'r', encoding='utf-8') as file:

    function_code      = ''
    capturing_function = False

    for line in file:

        # Check if the line is blank
        if line.strip() == '':

            # If the line is blank and we're capturing
            # a function, we've reached the end of the
            # current function; reset
            if capturing_function:
                functions.append(function_code.strip())
                function_code = ''
                capturing_function = False

        # If line isn't blank, start capturing function
        else:
            function_code += line
            capturing_function = True

    # If the file doesn't end in a blank line,
    # ensure the last function is saved.
    if capturing_function:
        functions.append(function_code.strip())

## Process Functions

In [6]:
# Initialize Agents & Tasks
agents = CrewAgents()
tasks  = CrewTasks()

# Create Agents
variable_name_agent = agents.variable_name_agent()
docstring_agent     = agents.docstring_agent()
qa_agent            = agents.quality_assurance_agent()
code_writer_agent   = agents.code_writer_agent()

# Process Functions
updated_functions = []

for function in functions:

    # Create tasks
    improve_variable_names = tasks.improve_variable_names(variable_name_agent, function)
    add_docstrings         = tasks.add_docstring(docstring_agent, improve_variable_names)
    review_code            = tasks.review_code(qa_agent, add_docstrings)
    write_code_to_file     = tasks.write_code(code_writer_agent, review_code)

    # Create crew & assign tasks
    crew = Crew(
        agents = [
            variable_name_agent,
            docstring_agent,
            qa_agent,
            code_writer_agent
        ],
        tasks = [
            improve_variable_names,
            add_docstrings,
            review_code,
            write_code_to_file
        ],
        verbose=True
    )

    # Start code-improvement process
    start_time = time.perf_counter()
    output = crew.kickoff()
    end_time = time.perf_counter()

    updated_functions.append(output.strip())

    print("################################")
    print("## Refactored Function Output ##")
    print("################################")
    print(f">> Time taken: {end_time - start_time:.2f} seconds <<\n")

 [DEBUG]: == Working Agent: Senior Python Developer
 [INFO]: == Starting Task: 
            Review the variable names in the Python code. Update
            the variable names so that they're accurate, concise, and
            easy to read and understand. Update the variable names only;
            don't change anything else about the functions, and do not
            add comments anywhere in the code.

            Python Function
            ---------------
            def f(n):
a, b = 0, 1
for _ in range(n):
    a, b = b, a + b
return a



> Entering new CrewAgentExecutor chain...
Thought: I'm glad this task is focused on improving variable names in Python functions. As a Senior Python Developer, I know how important it is to have descriptive and easy-to-understand variable names. It's crucial for maintaining code readability, reducing bugs, and making the code more maintainable.

Final Answer:

```
def fibonacci(n):
    previous_two_values = 0, 1
    for iteration in range(n):
     

2024-05-21 11:38:20,862 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I understand the importance of this task. I will carefully review the provided code and documentation to ensure that only the necessary information remains.

Final Answer:
```
def fibonacci(n):
    """Calculate the nth Fibonacci number.
    
    Args:
        n (int): The position of the Fibonacci number to calculate.
    
    Returns:
        int: The nth Fibonacci number.
    
    Example:
        >>> fibonacci(5)
        3
    
    Description:
        This function calculates the nth Fibonacci number using an iterative approach.
        It starts with the first two Fibonacci numbers, a and b, initialized as 0 and 1,
        and then iteratively calculates each subsequent number by updating the values
        of a and b. The final result is returned as the nth Fibonacci number.
    """
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a
```

> Finished chain.
 [DEBUG]: == [Senior Code Writer] Task output: ```
def fibonacci(n):
    """Calculate the nth Fibonac

In [7]:

def remove_triple_backticks(func_str):
    """Remove triple backticks from the beginning and end of a string."""
    return func_str.strip('`').strip()

# Check if directory exists; if not, create it
# Directory is relative to location of Jupyter Notebook
save_location_file_path = 'test_outputs'  # Removed leading slash for relative path
if not os.path.exists(save_location_file_path):
    os.makedirs(save_location_file_path)

# Create custom datetime output for filename
current_datetime = datetime.datetime.now().strftime('%y-%m-%d_%H-%M-%S') + '.txt'
output_filename  = 'functions_output_' + current_datetime
full_file_path   = os.path.join(save_location_file_path, output_filename)

# Write to the file
with open(full_file_path, 'w', encoding='utf-8') as f:
    for func in updated_functions:
        cleaned_func = remove_triple_backticks(func)
        f.write(cleaned_func + "\n\n")

print(f'Functions successfully written to file "{full_file_path}".')

Functions successfully written to file "test_outputs/functions_output_24-05-21_11-41-10.txt".
